Results shown in this notebook are promising, however, during further experiments, there were problems with stability of the results - the performance, due to the implementation provided in the package, in one run, was about 0.70, but in the other only about 0.45.

In [1]:
!pip3 install belt-nlp

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from belt_nlp.bert_classifier_with_pooling import BertClassifierWithPooling
from belt_nlp.bert_classifier_truncated import BertClassifierTruncated
from sklearn.metrics import balanced_accuracy_score
import torch

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
test_ns_data2=pd.read_csv("/source_repository/other_datasets_for_tests/news_sentiment_dataset/Sentiment_dataset.csv")
actaware_samples_gt=pd.read_csv("/source_repository/articles_categories_my_gt_2.csv", sep=';')
test_ns_data2['sentiment']=test_ns_data2['sentiment'].replace(1,2)

In [8]:
sentiment_mapping = {'negative': 0, 'positive': 2, 'neutral': 1}
actaware_samples_gt['Sentiment']=actaware_samples_gt['Sentiment'].map(sentiment_mapping)
actaware_samples_gt.head()

,Article,Category,Sentiment,Company,Certainity_company,Certainity_category,Certainity_sentiment
0,People work in the Amazon Fulfillment Center i...,human_employee_rights,0,Amazon,100,100,100
1,A federal agency is seeking to force Starbucks...,human_employee_rights,0,Starbucks,100,100,100
2,You might have seen a new energy drink on Amaz...,human_employee_rights,0,Amazon,100,90,100
3,The BBC's director-general has tried to calm t...,political_and_religious_views,0,BBC,100,100,100
4,Amazon is running a competition to give its br...,human_employee_rights,0,Amazon,100,100,90


In [9]:
actaware_samples_gt

,Article,Category,Sentiment,Company,Certainity_company,Certainity_category,Certainity_sentiment
0,People work in the Amazon Fulfillment Center i...,human_employee_rights,0,Amazon,100,100,100
1,A federal agency is seeking to force Starbucks...,human_employee_rights,0,Starbucks,100,100,100
2,You might have seen a new energy drink on Amaz...,human_employee_rights,0,Amazon,100,90,100
3,The BBC's director-general has tried to calm t...,political_and_religious_views,0,BBC,100,100,100
4,Amazon is running a competition to give its br...,human_employee_rights,0,Amazon,100,100,90
...,...,...,...,...,...,...,...
95,READ MOER: The Tesla killer? Toyota claims its...,other,1,Toyota,100,100,80
96,"Exterior view of the Siemens Forum, part of th...",corporate_transparency,2,Siemens,100,80,80
97,Companies are accused of greenwashing when the...,environment,0,no company,100,100,100
98,It has noted the doubling and quadrupling of t...,other,0,Amazon,100,100,90


In [5]:
ns_train, ns_test = train_test_split(test_ns_data2, test_size=0.2, random_state=42)
actaware_train, actaware_test=train_test_split(actaware_samples_gt, test_size=0.4, random_state=42)

X_train_ns=ns_train['text'].reset_index()['text']
y_train_ns=ns_train['sentiment'].reset_index()['sentiment']
X_test_ns=ns_test['text'].reset_index()['text']
y_test_ns=ns_test['sentiment'].reset_index()['sentiment']

X_train_act=actaware_train['Article'].reset_index()['Article']
y_train_act=actaware_train['Sentiment'].reset_index()['Sentiment']
X_test_act=actaware_test['Article'].reset_index()['Article']
y_test_act=actaware_test['Sentiment'].reset_index()['Sentiment']

In [8]:
y_train_act.unique(), y_test_act.unique()

(array([1, 0, 2]), array([0, 1, 2]))

In [36]:
MODEL_PARAMS = {
    "num_labels": 4,
    "batch_size": 16,
    "learning_rate": 5e-6,
    "epochs": 6,
    "chunk_size": 510,
    "stride": 510,
    "minimal_chunk_length": 510,
    "maximal_text_length": 510 * 4,
    "pooling_strategy": "mean",
    "device": "cuda",
    "pretrained_model_name_or_path": "nlptown/bert-base-multilingual-uncased-sentiment",
}

In [37]:
model_actaware_only = BertClassifierWithPooling(**MODEL_PARAMS)

In [38]:
#fine tune only on actaware
# model_actaware_only = BertClassifierWithPooling(**MODEL_PARAMS)

model_actaware_only.fit(list(X_train_act), list(y_train_act), epochs=6) #fit - fine-tune the model to the training set, use the list of raw texts and labels
classes = model_actaware_only.predict(list(X_test_act))
probabilities = model_actaware_only.predict_scores(list(X_test_act))

classes_train = model_actaware_only.predict(list(X_train_act))
accurate_train = sum(np.array(classes_train.cpu()) == np.array(list(y_train_act)))

balanced_accuracy_score(np.array(classes_train.cpu()), np.array(list(y_train_act)))

print(f"Test accuracy: {balanced_accuracy_score(np.array(list(y_test_act)),np.array(classes.cpu()))}.")
print(f"Train accuracy: {balanced_accuracy_score(np.array(classes_train.cpu()), np.array(list(y_train_act)))}.")

Test accuracy: 0.6296703296703297.
Train accuracy: 0.8931460195828013.


In [39]:
y_train_act.unique(), classes.unique(), classes_train.unique()

(array([1, 0, 2]),
 tensor([0, 1, 2], device='cuda:0'),
 tensor([0, 1, 2], device='cuda:0'))

In [32]:
MODEL_PARAMS = {
    "num_labels": 5,
    "batch_size": 16,
    "learning_rate": 5e-6,
    "epochs": 6,
    "device": "cuda",
    "pretrained_model_name_or_path": "nlptown/bert-base-multilingual-uncased-sentiment",
}

In [33]:
model_actaware_only_tr = BertClassifierTruncated(**MODEL_PARAMS)

In [34]:
#fine tune only on actaware

model_actaware_only_tr.fit(list(X_train_act), list(y_train_act), epochs=6) #fit - fine-tune the model to the training set, use the list of raw texts and labels
classes = model_actaware_only_tr.predict(list(X_test_act))
probabilities = model_actaware_only_tr.predict_scores(list(X_test_act))

classes_train = model_actaware_only_tr.predict(list(X_train_act))
accurate_train = sum(np.array(classes_train.cpu()) == np.array(list(y_train_act)))

balanced_accuracy_score(np.array(classes_train.cpu()), np.array(list(y_train_act)))

print(f"Test accuracy: {balanced_accuracy_score(np.array(list(y_test_act)),np.array(classes.cpu()))}.")
print(f"Train accuracy: {balanced_accuracy_score(np.array(classes_train.cpu()), np.array(list(y_train_act)))}.")

Test accuracy: 0.4897435897435898.
Train accuracy: 0.7188552188552189.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [6]:
MODEL_PARAMS = {
    "num_labels": 3,
    "batch_size": 16,
    "learning_rate": 5e-5,
    "epochs": 3,
    "chunk_size": 510,
    "stride": 510,
    "minimal_chunk_length": 510,
    "maximal_text_length": 510 * 4,
    "pooling_strategy": "mean",
    "device": "cuda",
    "pretrained_model_name_or_path": "nlptown/bert-base-multilingual-uncased-sentiment",
}

In [7]:
model_ns_only = BertClassifierWithPooling(**MODEL_PARAMS)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
#fine tune only on news

model_ns_only.fit(list(X_train_ns), list(y_train_ns), epochs=3) #fit - fine-tune the model to the training set, use the list of raw texts and labels
classes_ns = model_ns_only.predict(list(X_test_ns))
probabilities_ns = model_ns_only.predict_scores(list(X_test_ns))

classes_train_ns = model_ns_only.predict(list(X_train_ns))
accurate_train_ns = sum(np.array(classes_train_ns.cpu()) == np.array(list(y_train_ns)))

print(f"Test accuracy: {balanced_accuracy_score(np.array(list(y_test_ns)),np.array(classes_ns.cpu()))}.")
print(f"Train accuracy: {balanced_accuracy_score(np.array(classes_train_ns.cpu()), np.array(list(y_train_ns)))}.")

Test accuracy: 0.6907051282051282.
Train accuracy: 0.9761462696705536.
